# 1. Introduction  
I'll use this dataset with lots of face images of famous people to try to blend them two by two into one image only and imagine how such a mix would look like. An incredibly funny and troublesome exercise.  
We start by importing what is necessary for the project:

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import layers, optimizers
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib import image as mpimg
import cv2
import os

# 2. Creating dataframe  
I'm going to need a dataframe to better select and track who is being mixed with whom.  

In [2]:
def create_df(data_path):
    origin_path = None
    data = {}
    for dirpath, dirnames, filenames in os.walk(data_path):
        for filename in filenames:
            if filename.endswith('.jpg'):
                if origin_path is None:
                    origin_path = dirpath.replace(dirpath.split("/")[-1], "")
                name = dirpath.split("/")[-1]
                if name not in data:
                    data[name] = []
                data[name].append(filename)
    df = pd.DataFrame({name: pd.Series(files) for name, files in data.items()})
    return origin_path, df

In [3]:
# Set the path for the dataset
database_path = "/kaggle/input"

# Create the dataset
origin_path, df = create_df(database_path)

In [4]:
df

,daniel_craig,meryl_streep,kathy_bates,owen_wilson,faye_dunaway,liv_tyler,olivia_wilde,diane_keaton,sissy_spacek,david_niven,...,mel_gibson,colin_farrell,hugh_grant,gwyneth_paltrow,don_cheadle,julianne_moore,nicole_kidman,joseph_gordon_levitt,june_squibb,cary_grant
0,face_detected_6cac4193.jpg,face_detected_e855ce4f.jpg,face_detected_a8b5e604.jpg,face_detected_e0240648.jpg,face_detected_98e20f8a.jpg,face_detected_1539d7b3.jpg,face_detected_e02c1f7b.jpg,face_detected_9bacbc7a.jpg,face_detected_54f982fa.jpg,face_detected_5f2ecbaf.jpg,...,face_detected_8bb27593.jpg,face_detected_9dae2366.jpg,face_detected_27e123f1.jpg,face_detected_7959af1f.jpg,face_detected_a65ac956.jpg,face_detected_d2b8d69e.jpg,face_detected_fe885656.jpg,face_detected_f3ab0446.jpg,face_detected_7e103089.jpg,face_detected_d58941b6.jpg
1,face_detected_93d5f225.jpg,face_detected_988ecdd1.jpg,face_detected_bbdd278b.jpg,face_detected_cb569f41.jpg,face_detected_b881c2ed.jpg,face_detected_141cce95.jpg,face_detected_07073d87.jpg,face_detected_c9ada716.jpg,face_detected_be623819.jpg,face_detected_c68d6c40.jpg,...,face_detected_0c0db220.jpg,face_detected_f6a7f433.jpg,face_detected_65b828ab.jpg,face_detected_89d62ffa.jpg,face_detected_10b6a265.jpg,face_detected_447ae6b0.jpg,face_detected_e8e73eb2.jpg,face_detected_ff290eaa.jpg,face_detected_b11814cc.jpg,face_detected_8f347a6d.jpg
2,face_detected_bc456829.jpg,face_detected_2e042bdb.jpg,face_detected_0dbb7210.jpg,face_detected_2ae3e688.jpg,face_detected_121dc282.jpg,face_detected_5b0abca3.jpg,face_detected_3c6f2caa.jpg,face_detected_7721d371.jpg,face_detected_898f3305.jpg,face_detected_203ab776-ConvertImage.jpg,...,face_detected_ff098298.jpg,face_detected_762aff1a.jpg,face_detected_3dc7800f.jpg,face_detected_a88aee92.jpg,face_detected_6270c5ba.jpg,face_detected_8ccd83ba.jpg,face_detected_1f47ddb9.jpg,face_detected_20ce289a.jpg,face_detected_a7855fab.jpg,face_detected_1c6e30d5.jpg
3,face_detected_052aa363.jpg,face_detected_a4c3e404.jpg,face_detected_370c3d15.jpg,face_detected_03f58ed0.jpg,face_detected_e3ddb3fe.jpg,face_detected_3ede4158.jpg,face_detected_3457bf82.jpg,face_detected_6d120b25.jpg,face_detected_5494905b.jpg,face_detected_d398eabf.jpg,...,face_detected_567e6ab6.jpg,face_detected_2c2745fb.jpg,face_detected_f33e763d.jpg,face_detected_a72a8dde.jpg,face_detected_afe3e3ff.jpg,face_detected_6c7e1ca3.jpg,face_detected_43f7e994.jpg,face_detected_7375ef9a.jpg,face_detected_1809a967.jpg,face_detected_ccb44496.jpg
4,face_detected_754a709a.jpg,face_detected_d5ee12b0.jpg,face_detected_b8aafbb7.jpg,face_detected_0ea07c87.jpg,face_detected_58b820bf.jpg,face_detected_911b32af.jpg,face_detected_566d4787.jpg,face_detected_4216c384.jpg,face_detected_7532edd1.jpg,face_detected_a9fd0ee6-ConvertImage.jpg,...,face_detected_1f04032e.jpg,face_detected_1e7a5bec.jpg,face_detected_9c4456d7.jpg,face_detected_208c4cbe.jpg,face_detected_86a1997b.jpg,face_detected_ae391472.jpg,face_detected_1a996194.jpg,face_detected_fd091115.jpg,face_detected_0aae195d.jpg,face_detected_600f19df.jpg
5,face_detected_d6732188.jpg,face_detected_7e2345e1.jpg,face_detected_6be776a7.jpg,face_detected_87c5da7c.jpg,face_detected_fcfb302d.jpg,face_detected_c2d37a18.jpg,face_detected_6b60fd72.jpg,face_detected_f8709d90.jpg,face_detected_9ec808f6.jpg,face_detected_8d2d5dd8-ConvertImage.jpg,...,face_detected_e607b786.jpg,face_detected_a45a72de.jpg,face_detected_61bc5222.jpg,face_detected_cfc5ff08.jpg,face_detected_447ab545.jpg,face_detected_162fa8c3.jpg,face_detected_364e93f2.jpg,face_detected_5f6bffa6.jpg,face_detected_92b30b60.jpg,face_detected_0fb422f2.jpg
6,face_detected_d93dbbde.jpg,face_detected_86ece0c1.jpg,face_detected_5eda01f3.jpg,face_detected_02be815a.jpg,face_detected_550cd7f5.jpg,NaN,face_detected_33efd9de.jpg,face_detected_c2644130.jpg,face_detected_e3003d1f.jpg,face_detected_1bd920cf.jpg,...,face_detected_b2353484.jpg,face_detected_679e44e8.jpg,face_detected_cf21de5e.jpg,face_detected_769dc54c.jpg,face_detected_082a6f19.jpg,face_detected_1e84f227.jpg,face

For this first project I'll use only those celebrities who has at least 25 images inside this dataset.  
Maybe we can do a second and third part later with other famous people.  
(It really depends more on the free available GPUs and RAM out there than anything else)

In [5]:
def filter_df(df):
    for col in df:
        if df[col].isnull().sum() > 4: # Greater the number, the more people are included in the sample
            df.drop(col, axis=1, inplace=True)
    df.dropna(inplace=True) # Used to make everyone have the same number of images to train
    return df

In [6]:
# Take a sample from the original images
df = filter_df(df)

In [7]:
df.shape

(25, 4)

Sorting the columns alphabetically to prevent randomness.

In [8]:
df.sort_index(axis=1, inplace=True)

In [9]:
df.columns

Index(['adam_sandler', 'amanda_seyfried', 'angelina_jolie', 'jesse_eisenberg'], dtype='object')

In the future, if we need to drop one or some particular columns, it can be easily done with the following command, for example:

In [10]:
#df.drop(df.columns[[0]], axis=1, inplace=True)

# 3. Preparing images  
After selecting the images for this project, we are going to resize them to 160 x 160 pixels, and finally save these into a new folder.

In [11]:
# Load image from file
def open_image(path, col, row, df):
    source_path = os.path.join(path, col, str(df[col][row]))
    img = cv2.imread(source_path)
    return img

In [12]:
# Resize to desired size
def resize_image(img):
    img = cv2.resize(img, img_size)
    # Convert BGR color space to RGB color space
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

In [13]:
# Save the images to disk
def save_image(path, col, row, df, img):
    destiny_path = os.path.join(path, col, df[col][row])
    os.makedirs(os.path.dirname(destiny_path), exist_ok=True)
    # Save the image using matplotlib
    mpimg.imsave(destiny_path, img)

In [14]:
def resize_database(df):
    for col in df.columns:
        for row in df.index:
            img = open_image(origin_path, col, row, df)
            img = resize_image(img)
            save_image(resize_path, col, row, df, img)

In [15]:
# Set the image size
img_size = (160, 160)

# Set the path for the directory to save the images
resize_path = "/kaggle/working/sample"

# Make a proper sample to train
resize_database(df)

# 4. Making GAN models  
A Generative Adversarial Networks consist in a generator model and a discriminator model.  
- The latter one trains in real images to learn how they look like and then classify them, for instance, either as true faces or as fake ones.  
- And the former one generates random images trying to fully mimic them to the point of fooling the discriminator as if they were true images.  

Therefore it's really a cat and rat game. And here are the models and their losses functions:

In [16]:
# Define the generator model
def make_generator_model(latent_dim):
    model = tf.keras.Sequential()

    # Input layer
    model.add(layers.Dense(10*10*512, use_bias=False, input_shape=(latent_dim,)))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())
    
    model.add(layers.Reshape((10, 10, 512)))
    assert model.output_shape == (None, 10, 10, 512)

    # First convolutional transpose layer
    model.add(layers.Conv2DTranspose(256, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    assert model.output_shape == (None, 20, 20, 256)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    # Second convolutional transpose layer
    model.add(layers.Conv2DTranspose(128, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    assert model.output_shape == (None, 40, 40, 128)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    # Third convolutional transpose layer
    model.add(layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    assert model.output_shape == (None, 80, 80, 64)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    # Fourth convolutional transpose layer
    model.add(layers.Conv2DTranspose(32, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    assert model.output_shape == (None, 160, 160, 32)
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    # Output layer
    model.add(layers.Conv2DTranspose(3, (5, 5), strides=(1, 1), padding='same', use_bias=False, activation='tanh'))
    assert model.output_shape == (None, 160, 160, 3)

    return model

In [17]:
# Define the discriminator model
def make_discriminator_model(img_shape):
    model = tf.keras.Sequential()
    
    # First convolutional layer
    model.add(layers.Conv2D(64, (5, 5), strides=(2, 2), padding='same',
                            input_shape=img_shape))
    assert model.output_shape == (None, img_shape[0]//2, img_shape[1]//2, 64)
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))
    
    # Second convolutional layer
    model.add(layers.Conv2D(128, (5, 5), strides=(2, 2), padding='same'))
    assert model.output_shape == (None, img_shape[0]//4, img_shape[1]//4, 128)
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))
    
    # Third convolutional layer
    model.add(layers.Conv2D(256, (5, 5), strides=(2, 2), padding='same'))
    assert model.output_shape == (None, img_shape[0]//8, img_shape[1]//8, 256)
    model.add(layers.LeakyReLU())
    model.add(layers.Dropout(0.3))
    
    # Output layer
    model.add(layers.Flatten())
    assert model.output_shape == (None, (img_shape[0]//8)*(img_shape[1]//8)*256)
    model.add(layers.Dense(1))

    return model

In [18]:
# Define the loss functions for the generator and discriminator
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    cross = cross_entropy(tf.ones_like(fake_output), fake_output)
    return cross

Since we are already getting the images two by two for training, I'll also generate their mixes soon after it in order to keep track of their origin.

In [19]:
# Define the training loop for the GAN
def train_gan(generator, discriminator,
              generator_loss, discriminator_loss,
              generator_optimizer, discriminator_optimizer,
              dataset, latent_dim, epochs):
    
    for epoch in range(epochs):
        for images in dataset:
            # Generate random noise vector
            noise = tf.random.normal([batch_size, latent_dim])

            with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
                
                # Generate fake images from the noise vector
                generated_images = generator(noise, training=True)

                # Evaluate the discriminator on both real and fake images
                real_output = discriminator(images, training=True)
                fake_output = discriminator(generated_images, training=True)

                # Calculate the loss for the generator and discriminator
                gen_loss = generator_loss(fake_output)
                disc_loss = discriminator_loss(real_output, fake_output)

            # Compute the gradients for both generator and discriminator
            gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
            gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

            # Apply the gradients to the generator and discriminator
            generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
            discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

        print(f"Epoch {epoch + 1} complete")

In [20]:
# Define the generating loop for the mixed images
def generate_images(image1_path, image2_path, n_images):
    for i in range(n_images):
        # Generate random noise vector
        noise = tf.random.normal([batch_size, latent_dim])
        
        # Use the generator model to generate a fake image from the noise vector
        mixed_image = generator.predict(noise)[0]
        
        # Save the mixed image
        output_path = os.path.join("/kaggle/working/", "mixed_images", f"{image1_path}_{image2_path}", f"{i+1}_mixed_image.jpg")
        os.makedirs(os.path.dirname(output_path), exist_ok=True)
        tf.keras.preprocessing.image.save_img(output_path, mixed_image)
        
        print(f"{i+1} mixed image(s)")

In [21]:
def init_models(latent_dim, images):
    generator = make_generator_model(latent_dim)
    discriminator = make_discriminator_model(images.shape[1:])
    return generator, discriminator

In [22]:
def prepare_dataset(images):
    imageset = np.array(images)
    dataset = tf.data.Dataset.from_tensor_slices(imageset)
    dataset = dataset.shuffle(buffer_size=1024).batch(batch_size)
    return imageset, dataset

In [23]:
# Normalize the pixels to values between -1 and 1
def normal_image(img):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = (img - 127.5) / 127.5
    return img

# 5. Running everything  
In our loop we have to prevent pointless mixes like:  
- one person and itself;  
- same pairs of people.  

I recommend training at least 2000 epochs with free available GPU and RAM and maybe twice or thrice that if you can afford paying - here on Kaggle I'll run only once as an example.  
It takes a long time, so take your chance and generate as much images as you want in the process - here on Kaggle I'll generate only one as an example.

In [24]:
# Set the batch size and the latent dim used for generating random images
batch_size = 128
latent_dim = 150

In [25]:
mixed_pairs = set()
for name1 in df.columns:
    for name2 in df.columns:
        # Check if it's the same person
        if name1 == name2:
            continue
        
        # Check if it's the same pair of people
        pair_key = frozenset([name1, name2])
        if pair_key in mixed_pairs:
            continue
        mixed_pairs.add(pair_key)
        
        # Load the selected images
        images = []
        for row in df.index:
            img1 = open_image(resize_path, name1, row, df)
            img2 = open_image(resize_path, name2, row, df)
            images.append(normal_image(img1))
            images.append(normal_image(img2))
        
        # Initialize dataset and models
        images, dataset = prepare_dataset(images)
        generator, discriminator = init_models(latent_dim, images)
        
        # Define the optimizer for the generator and discriminator
        generator_optimizer = optimizers.Adam(learning_rate=0.0002, beta_1=0.5, beta_2=0.9)
        discriminator_optimizer = optimizers.Adam(learning_rate=0.0002, beta_1=0.5, beta_2=0.9)
        
        # Train the GAN on the dataset
        train_gan(generator, discriminator,
                  generator_loss, discriminator_loss,
                  generator_optimizer, discriminator_optimizer,
                  dataset, latent_dim, epochs=1) # You might want to train thousands of epochs
        
        # Generate mixed images using the generator model
        generate_images(name1, name2, n_images=1) # You might want to generate dozens of images
        
        # Prevent memory overload
        tf.keras.backend.clear_session()
        
        #break # Use this break if you only want one pair mixed
    print(f"End of {name1} folder")
    #break # Use this break if you only want one person mixed with everyone else

Epoch 1 complete
4/4 [==============================] - 7s 2s/step
1 mixed image(s)
Epoch 1 complete
4/4 [==============================] - 7s 2s/step
1 mixed image(s)
Epoch 1 complete
4/4 [==============================] - 7s 2s/step
1 mixed image(s)
End of adam_sandler folder
Epoch 1 complete
4/4 [==============================] - 7s 2s/step
1 mixed image(s)
Epoch 1 complete
4/4 [==============================] - 7s 2s/step
1 mixed image(s)
End of amanda_seyfried folder
Epoch 1 complete
4/4 [==============================] - 7s 2s/step
1 mixed image(s)
End of angelina_jolie folder
End of jesse_eisenberg folder


# 6. Conclusion  
In order to see the results of this project you can enter the site: https://starsmixer.onrender.com  
I hope to bring more mixes in time to come and inspire others to do the same just for the fun of it.